# Sentiment analysis of show reviews

The goal of this analysis is to ...
* Get hands-on experience with packages and tools for analysing Russian language (natasha, nltk, spacy, rnnmorph, pymorphy2)
* Investigate available pre-trained models for Russian language (wor2vec, fasttext, navec, models from sber, deeppavlov and others)
* Learn how to finetune BERT-like models

## Imports

In [1]:
import gc
import os
import re
import sys
import warnings
from typing import List, Tuple

import dateparser
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

tqdm.pandas()

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
SEED = 42

## Data

### Loading data

In [3]:
%%time
reviews_df = pd.read_parquet("../../data/mt_reviews.parquet")
reviews_df.shape

CPU times: total: 3.77 s
Wall time: 3.98 s


(206737, 9)

### Dataset overview

In [4]:
reviews_df.sample(n=10, random_state=SEED)

show_id  user_id    type            datetime sentiment  \
196236   257386    28525  series 2010-08-27 11:23:00      good   
128582   688832    44953   movie 2015-02-20 23:19:00       bad   
159673      349    33910   movie 2018-04-15 21:04:00      good   
109244   686898    44065   movie 2019-11-09 11:58:00   neutral   
92610     61455    66782   movie 2017-11-27 18:52:00      good   
43727    491724    44563   movie 2012-01-28 23:18:00      good   
50195    102130    67145   movie 2009-08-17 11:56:00      good   
43651    491724    66283   movie 2012-02-19 15:33:00       bad   
48775      7226     7905   movie 2014-02-20 03:44:00      good   
75262       458    33255   movie 2013-06-22 21:14:00      good   

                                        subtitle  \
196236                             Вечный город.   
128582                 50 оттенков разочарования   
159673    Господи, спасибо, что не пронесло мимо   
109244                   Что же стало с клоуном?   
92610               Они отказываются подчиняться   
43727                            Жестокая правда   
50195   Преодолеть 2 года жизни, что встретиться   
43651        Мужчины, которые ненавидели женщин.   
48775                                       None   
75262                       Тайна закрытой двери   

                                                   review  \
196236  К истории Древнего мира у меня отношение особо...   
128582  Говорю сразу, книги читала все, да и по нескол...   
159673  Есть два типа фильмов, мой друг. Одни ты прост...   
109244  Итак, в первую очередь хотелось бы отметить то...   
92610   Автора этого замечательного фильма, Джосса Уэд...   
43727   Финчер снова нас поразил, он всегда нас поража...   
50195   Слышал о фильме много, и в основном положитель...   
43651   Я попробовала рассматривать этот фильм с двух ...   
48775   «Догвилль» - это один из тех редких фильмов, п...   
75262   Я имела счастье смотреть этот мультфильм в кин...   

        usefulness_ratio_transformed  score  
196236                            40    NaN  
128582                             5    3.0  
159673                             4    NaN  
109244                             2    6.5  
92610                              5    NaN  
43727                              7   10.0  
50195                             11   10.0  
43651                              3    6.0  
48775                              3   10.0  
75262                              6   10.0

### Looking at reviews

In [6]:
for review in reviews_df["review"].values[:10]:
    print(review.replace("<p>", "\n"))
    print("\n")

Варкрафт снял отличный режиссёр Данкан Джонс, который до этого создал великолепные Луна 2112 и Исходный код.
У данного кино по мотивам популярнейшей серии одноименных компьютерных игр непростая судьба. На зарубежных ресурсах фильм имеет в основном негативные отзывы, среди российских зрителей неожиданно позитивные.
Сюжет рассказывает как некая орда с очевидно ближневосточных по нашему восприятию земель, где закончились ресурсы и стало невыносимо жить, массово пытается переселиться к более цивилизованным людям. Люди этому активно сопротивляются. Никто поначалу даже не задумывается о межкультурном диалоге. Проблемы пытаются решить силой. Меж тем вопреки всему отдельные представили разных народов сумели найти общий язык и вместе решили избавиться от источника всех бед - скверны, которая поражает и развращает по обе стороны охранителей-пропагандистов - магов, наделённых властью. Именно с помощью скверны высасываются силы из простых и слабых существ.
По ходу истории, кстати, выясняется, что 

In the previous step I've removed the scores from the reviews so it is now safe to continue with baseline model creation.

### Selecting needed columns

For baseline model we're interested only in `sentiment` and `review_body` columns

In [7]:
df = reviews_df[["sentiment", "review"]]

In [8]:
del reviews_df
gc.collect()

975

### Splitting the data

In [9]:
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=SEED, stratify=df["sentiment"]
)
train_df.shape, test_df.shape

((165389, 2), (41348, 2))

In [10]:
train_df["sentiment"].value_counts(normalize=True)

good       0.720332
neutral    0.149974
bad        0.129694
Name: sentiment, dtype: float64

In [11]:
test_df["sentiment"].value_counts(normalize=True)

good       0.720325
neutral    0.149971
bad        0.129704
Name: sentiment, dtype: float64

## Modelling

### Hyperparameter Investigation

#### `lowercase`

In [16]:
%%time
# withour lowercase
vectorizer = CountVectorizer(lowercase=False)
vectors_wo_lowercase = vectorizer.fit_transform(train_df["review_body"])
print(
    f"The size of the train dataset is {vectors_wo_lowercase.shape} with lowercase turned off"
)

The size of the train dataset is (165389, 753898) with lowercase turned off
CPU times: total: 35.5 s
Wall time: 35.5 s


In [17]:
%%time
# lowercase
vectorizer = CountVectorizer()
vectors_w_lowercase = vectorizer.fit_transform(train_df["review_body"])
print(
    f"The size of the train dataset is {vectors_w_lowercase.shape} with lowercase turned on"
)

The size of the train dataset is (165389, 642874) with lowercase turned on
CPU times: total: 38.2 s
Wall time: 38.2 s


In [18]:
vectors_wo_lowercase.shape[1] - vectors_w_lowercase.shape[1]

111024

The difference in vocabulary size without making all characters lowercase and with lowercase is more than 100 000, so we better stick to lowercase 

#### `max_df` and `min_df`

In [19]:
vectorizer.get_feature_names_out()[:50]

array(['00', '000', '0000', '00000', '000000',
       '000000000000000000попкорн000000000000', '000000000000001',
       '000000000000на', '00000000000во', '00000000000данной',
       '00000000000есть000000000000000',
       '00000000000есть000000000000000000', '0000000000жевать',
       '0000000000ненавижу00000000', '00000000016', '000000000надо',
       '000000000разговаривать0000000000', '000000001',
       '00000000визуальная', '00000001', '000001', '00000громко',
       '00000точек', '00001', '0001', '0002', '000доктора', '000какой',
       '000км', '000косметические', '000теряются', '001', '002', '003',
       '00381', '006', '007', '00в', '00вых', '00е', '00м', '00по', '00с',
       '00седьмого', '00х', '00ые', '00ых', '01', '0100', '011'],
      dtype=object)

We can see that if we do not limit the vocabulary, we will have very infrequent words, so we better do it.  
For that we have to choose the `min_df` and `max_df` thresholds.

In [20]:
%%time
# min_df
vectorizer = CountVectorizer(min_df=0.8)
vectors = vectorizer.fit_transform(train_df["review_body"])
vectors.shape

CPU times: total: 34.7 s
Wall time: 34.7 s


(165389, 8)

In [21]:
vectorizer.get_feature_names_out()

array(['из', 'как', 'на', 'не', 'но', 'то', 'что', 'это'], dtype=object)

These words are in the 80% of all reviews and it is understandable.  
`из` is there because many reviews contain a score like `7.6 из 10` and other words are just common.  

In [22]:
%%time
# min_df
MIN_DF = 0.01
vectorizer = CountVectorizer(min_df=MIN_DF)
vectors = vectorizer.fit_transform(train_df["review_body"])
print(
    f"The size of the train dataset is {vectors.shape} with lowercase turned on and min_df={MIN_DF}"
)

The size of the train dataset is (165389, 3352) with lowercase turned on and min_df=0.01
CPU times: total: 35.1 s
Wall time: 35.1 s


In [23]:
vectorizer.get_feature_names_out()[:50]

array(['10', '100', '11', '12', '13', '15', '16', '17', '18', '20',
       '2012', '21', '30', '3d', '40', '50', '60', '70', '80', '90', 'dc',
       'marvel', 'of', 'the', 'абсолютно', 'аватар', 'автор', 'автора',
       'авторов', 'авторы', 'агент', 'аж', 'актер', 'актера', 'актерам',
       'актерами', 'актерах', 'актеров', 'актером', 'актерская',
       'актерский', 'актерского', 'актерской', 'актерскую', 'актеры',
       'актриса', 'актрисы', 'актёр', 'актёра', 'актёров'], dtype=object)

In [69]:
%%time
# ngram_range
NGRAM_RANGE = (1, 3)
vectorizer = CountVectorizer(ngram_range=NGRAM_RANGE, min_df=MIN_DF)
train_vectors = vectorizer.fit_transform(train_df["review_body"])
print(
    f"The size of the train dataset is {vectors.shape} with lowercase turned on and min_df={MIN_DF} and ngram_range={NGRAM_RANGE}"
)

The size of the train dataset is (165389, 4840) with lowercase turned on and min_df=0.01 and ngram_range=(1, 3)
CPU times: total: 4min 28s
Wall time: 10min 16s


In [70]:
vectorizer.get_feature_names_out()[:50]

array(['10', '10 за', '10 из', '10 из 10', '10 лет', '100', '11', '12',
       '13', '15', '16', '17', '18', '20', '2012', '21', '30', '3d', '40',
       '50', '60', '70', '80', '90', 'dc', 'marvel', 'of', 'the',
       'абсолютно', 'абсолютно все', 'абсолютно не', 'аватар', 'автор',
       'автора', 'авторов', 'авторы', 'агент', 'аж', 'актер', 'актера',
       'актерам', 'актерами', 'актерах', 'актеров', 'актером',
       'актерская', 'актерская игра', 'актерский', 'актерский состав',
       'актерского'], dtype=object)

In [71]:
test_vectors = vectorizer.transform(test_df["review_body"])

### Label Encoding

In [73]:
le = LabelEncoder()
train_labels = le.fit_transform(train_df["sentiment"])
test_labels = le.transform(test_df["sentiment"])

In [ ]:
%%time
svc = svm.SVC()
svc.fit(train_vectors, train_labels)

In [ ]:
f1_score(test_labels, svc.predict(test_vectors))

### Tf-Idf for `review_body`

In [24]:
vectorizer_params = {
    "ngram_range": (1, 2),
    "max_features": 10000,
    "tokenizer": lambda s: s.split(),
}
vectorizer_article = TfidfVectorizer(**vectorizer_params)

In [25]:
%%timeit
X_train_review = vectorizer_article.fit_transform(train_df["review_body"])


KeyboardInterrupt



In [ ]:
%%timeit
X_test_review = vectorizer_article.transform(test_df["review_body"])

### Training LogReg

In [20]:
log_reg = LogisticRegression(random_state=SEED)